In [46]:
%qtconsole
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import numpy as np

In [53]:
def simplexit(A,b,c, init, debug=True, expectNullOptimum=False):
    """
    Implements the standard matrix form of the simplex algorithm

    Hugues Talbot 2018, from an initial version in R ca. 2009.
    """
    iter = 0
    dim=A.shape[0]
    nbvar=A.shape[1]
    lastx=0
    lastxl=0
    sol=None
    IBVmem=None # to detect cycles

    vars=np.arange(0,nbvar) # indices start at 0
    IBV=init # Initial basis vector
    while (True):
        if debug:
            print("*** Iteration: ", iter)
        NBV = np.setdiff1d(vars,IBV) # Set complement -> non basis variables
        if debug:
            print("    IBV = ", IBV)
            print("    NBV = ", NBV)

        B=A[:,IBV]
        E=A[:,NBV]
        cb = c[IBV]
        ce = c[NBV]

        if debug:
            print("    B = \n", B)
            print("    E = \n", E)

        try:
            iB = np.linalg.inv(B)
        except:
            print("B is not invertible, error")
            break

        if debug:
            print("    iB = \n", iB)

        bbar = iB.dot(b)
        if (np.min(bbar) < -1e-7):
            print("bbar is not positive, not a feasible solution")
            break

        if debug:
            print("    bbar= ", bbar)

        cebart = ce.T - cb.T.dot(iB).dot(E)

        if debug:
            print("  reduced costs", cebart)

        if (np.min(cebart) >= 0):
            print(" optimum found!")
            sol = (IBV, bbar)
            print("IBV = ", IBV)
            print("Bbar= ", bbar)
            if ((np.min(cebart) > 0) or expectNullOptimum):
                break
            if (IBVmem is None):
                IBVmem = IBV
            else:
                if (IBV == IBVmem):
                    print("Cycle detected\n")
                    break

        l = np.argmin(cebart) # may not be unique
        P = iB.dot(A[:,NBV[l]])
        if (debug):
            print("P = ", P)
        if (np.max(P) <= 0):
            print("Solution in unbounded!")
            break
        ratios = bbar/P ## elementwise division
        if (debug):
            print("Ratios = ", ratios)
        ## Ignore negative members f P
        negP = P < 0
        ratios[negP] = np.inf ## this way they will not be selected as the min
        nanP = np.isnan(ratios)
        ratios[nanP] = np.inf ## nan are eliminated as well
        if (debug):
            print("Fixed ratios = ", ratios)
        xl = np.min(ratios)
        j = np.argmin(ratios)
        lastx = xl
        lastxl = NBV[l]
        if (debug):
            print("Variable (", IBV[j],") in the basis replaced by variable (",NBV[l],")\n",sep="")
            print("xl = ",xl)

        IBV[j] = NBV[l]
        if (debug):
            print("IBV = ", IBV)

        iter += 1
        ## end while
        
    return sol

In [54]:
## exercice 2, TD1 (production de A,B,C liée)
def exampleII(debug=True):
    # A <- matrix(data =c(-1, 0, 1, 2, -1, 2, 0, 1, 3, 1, 0, 0, 0, 1, 0, 0, 0, 1), ncol=6, nrow=3)
    A = np.array([[-1,  2, 0, 1, 0, 0], 
                  [ 0, -1, 1, 0, 1, 0], 
                  [ 1,  2, 3, 0, 0, 1]])
    b = np.array([  0,  0, 35])
    C = np.array([-10,-36,-44, 0, 0, 0])
    IBV = np.array([3,4,5]) # index of the IBV
    sol = simplexit(A,b,C,IBV,debug)
    
    # calcul optimum
    z = C[sol[0]].dot(sol[1])
    return(z)


In [55]:
exampleII()

*** Iteration:  0
    IBV =  [3 4 5]
    NBV =  [0 1 2]
    B = 
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
    E = 
 [[-1  2  0]
 [ 0 -1  1]
 [ 1  2  3]]
    iB = 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
    bbar=  [ 0.  0. 35.]
  reduced costs [-10. -36. -44.]
P =  [0. 1. 3.]
Ratios =  [        nan  0.         11.66666667]
Fixed ratios =  [        inf  0.         11.66666667]
Variable (4) in the basis replaced by variable (2)

xl =  0.0
IBV =  [3 2 5]
*** Iteration:  1
    IBV =  [3 2 5]
    NBV =  [0 1 4]
    B = 
 [[1 0 0]
 [0 1 0]
 [0 3 1]]
    E = 
 [[-1  2  0]
 [ 0 -1  1]
 [ 1  2  0]]
    iB = 
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0. -3.  1.]]
    bbar=  [ 0.  0. 35.]
  reduced costs [-10. -80.  44.]
P =  [ 2. -1.  5.]
Ratios =  [ 0. -0.  7.]
Fixed ratios =  [ 0. inf  7.]
Variable (3) in the basis replaced by variable (1)

xl =  0.0
IBV =  [1 2 5]
*** Iteration:  2
    IBV =  [1 2 5]
    NBV =  [0 3 4]
    B = 
 [[ 2  0  0]
 [-1  1  0]
 [ 2  3  1]]
    E = 
 [[-1  1  0]
 [ 0  0  1]
 [ 1  0  0]]
  

simplexe.py:84: RuntimeWarning: invalid value encountered in divide
  ratios = bbar/P ## elementwise division


-500.0